In [1]:
import os
import json
import cv2

In [2]:
root_dir = '/media/commaai-03/Data/workdata/baodi/weighter/rs07/oneBox_twoHead/'
alls = os.listdir(root_dir)

allimgs = [img for img in alls 
            if img.split('.')[-1].lower() in ['jpg', 'jpeg', 'png']]

alljsons = [file for file in alls
             if file.split('.')[-1].lower() in ['json']]

In [3]:
import numpy as np
from scipy.ndimage import uniform_filter, gaussian_filter
def ssim(X, Y, win_size=None, gradient=False,
         data_range=None, multichannel=False, gaussian_weights=False,
         full=False, dynamic_range=None, **kwargs):
    """Compute the mean structural similarity index between two images.
    Parameters
    ----------
    X, Y : ndarray
        Image.  Any dimensionality.
    win_size : int or None
        The side-length of the sliding window used in comparison.  Must be an
        odd value.  If `gaussian_weights` is True, this is ignored and the
        window size will depend on `sigma`.
    gradient : bool, optional
        If True, also return the gradient.
    data_range : int, optional
        The data range of the input image (distance between minimum and
        maximum possible values).  By default, this is estimated from the image
        data-type.
    multichannel : bool, optional
        If True, treat the last dimension of the array as channels. Similarity
        calculations are done independently for each channel then averaged.
    gaussian_weights : bool, optional
        If True, each patch has its mean and variance spatially weighted by a
        normalized Gaussian kernel of width sigma=1.5.
    full : bool, optional
        If True, return the full structural similarity image instead of the
        mean value.
    Other Parameters
    ----------------
    use_sample_covariance : bool
        if True, normalize covariances by N-1 rather than, N where N is the
        number of pixels within the sliding window.
    K1 : float
        algorithm parameter, K1 (small constant, see [1]_)
    K2 : float
        algorithm parameter, K2 (small constant, see [1]_)
    sigma : float
        sigma for the Gaussian when `gaussian_weights` is True.
    Returns
    -------
    mssim : float
        The mean structural similarity over the image.
    grad : ndarray
        The gradient of the structural similarity index between X and Y [2]_.
        This is only returned if `gradient` is set to True.
    S : ndarray
        The full SSIM image.  This is only returned if `full` is set to True.
    Notes
    -----
    To match the implementation of Wang et. al. [1]_, set `gaussian_weights`
    to True, `sigma` to 1.5, and `use_sample_covariance` to False.
    References
    ----------
    .. [1] Wang, Z., Bovik, A. C., Sheikh, H. R., & Simoncelli, E. P.
       (2004). Image quality assessment: From error visibility to
       structural similarity. IEEE Transactions on Image Processing,
       13, 600-612.
       https://ece.uwaterloo.ca/~z70wang/publications/ssim.pdf,
       DOI:10.1.1.11.2477
    .. [2] Avanaki, A. N. (2009). Exact global histogram specification
       optimized for structural similarity. Optical Review, 16, 613-621.
       http://arxiv.org/abs/0901.0065,
       DOI:10.1007/s10043-009-0119-z
    """
    _integer_types = (np.byte, np.ubyte,          # 8 bits
                      np.short, np.ushort,        # 16 bits
                      np.intc, np.uintc,          # 16 or 32 or 64 bits
                      np.int_, np.uint,           # 32 or 64 bits
                      np.longlong, np.ulonglong)  # 64 bits
    _integer_ranges = {t: (np.iinfo(t).min, np.iinfo(t).max)
                       for t in _integer_types}
    dtype_range = {np.bool_: (False, True),
                   np.bool8: (False, True),
                   np.float16: (-1, 1),
                   np.float32: (-1, 1),
                   np.float64: (-1, 1)}
    dtype_range.update(_integer_ranges)
    
    if not X.dtype == Y.dtype:
        raise ValueError('Input images must have the same dtype.')

    if not X.shape == Y.shape:
        raise ValueError('Input images must have the same dimensions.')

    if dynamic_range is not None:
        warn('`dynamic_range` has been deprecated in favor of '
             '`data_range`. The `dynamic_range` keyword argument '
             'will be removed in v0.14', skimage_deprecation)
        data_range = dynamic_range

    if multichannel:
        # loop over channels
        args = dict(win_size=win_size,
                    gradient=gradient,
                    data_range=data_range,
                    multichannel=False,
                    gaussian_weights=gaussian_weights,
                    full=full)
        args.update(kwargs)
        nch = X.shape[-1]
        mssim = np.empty(nch)
        if gradient:
            G = np.empty(X.shape)
        if full:
            S = np.empty(X.shape)
        for ch in range(nch):
            ch_result = compare_ssim(X[..., ch], Y[..., ch], **args)
            if gradient and full:
                mssim[..., ch], G[..., ch], S[..., ch] = ch_result
            elif gradient:
                mssim[..., ch], G[..., ch] = ch_result
            elif full:
                mssim[..., ch], S[..., ch] = ch_result
            else:
                mssim[..., ch] = ch_result
        mssim = mssim.mean()
        if gradient and full:
            return mssim, G, S
        elif gradient:
            return mssim, G
        elif full:
            return mssim, S
        else:
            return mssim

    K1 = kwargs.pop('K1', 0.01)
    K2 = kwargs.pop('K2', 0.03)
    sigma = kwargs.pop('sigma', 1.5)
    if K1 < 0:
        raise ValueError("K1 must be positive")
    if K2 < 0:
        raise ValueError("K2 must be positive")
    if sigma < 0:
        raise ValueError("sigma must be positive")
    use_sample_covariance = kwargs.pop('use_sample_covariance', True)

    if win_size is None:
        if gaussian_weights:
            win_size = 11  # 11 to match Wang et. al. 2004
        else:
            win_size = 7   # backwards compatibility

    if np.any((np.asarray(X.shape) - win_size) < 0):
        raise ValueError(
            "win_size exceeds image extent.  If the input is a multichannel "
            "(color) image, set multichannel=True.")

    if not (win_size % 2 == 1):
        raise ValueError('Window size must be odd.')

    if data_range is None:
        dmin, dmax = dtype_range[X.dtype.type]
        data_range = dmax - dmin

    ndim = X.ndim

    if gaussian_weights:
        # sigma = 1.5 to approximately match filter in Wang et. al. 2004
        # this ends up giving a 13-tap rather than 11-tap Gaussian
        filter_func = gaussian_filter
        filter_args = {'sigma': sigma}

    else:
        filter_func = uniform_filter
        filter_args = {'size': win_size}

    # ndimage filters need floating point data
    X = X.astype(np.float64)
    Y = Y.astype(np.float64)

    NP = win_size ** ndim

    # filter has already normalized by NP
    if use_sample_covariance:
        cov_norm = NP / (NP - 1)  # sample covariance
    else:
        cov_norm = 1.0  # population covariance to match Wang et. al. 2004

    # compute (weighted) means
    ux = filter_func(X, **filter_args)
    uy = filter_func(Y, **filter_args)

    # compute (weighted) variances and covariances
    uxx = filter_func(X * X, **filter_args)
    uyy = filter_func(Y * Y, **filter_args)
    uxy = filter_func(X * Y, **filter_args)
    vx = cov_norm * (uxx - ux * ux)
    vy = cov_norm * (uyy - uy * uy)
    vxy = cov_norm * (uxy - ux * uy)

    R = data_range
    C1 = (K1 * R) ** 2
    C2 = (K2 * R) ** 2

    A1, A2, B1, B2 = ((2 * ux * uy + C1,
                       2 * vxy + C2,
                       ux ** 2 + uy ** 2 + C1,
                       vx + vy + C2))
    D = B1 * B2
    S = (A1 * A2) / D

    # to avoid edge effects will ignore filter radius strip around edges
    pad = (win_size - 1) // 2

    # compute (weighted) mean of ssim
    from numpy.lib.arraypad import _as_pairs
    ar = np.array(S, copy=False)
    crops = _as_pairs(pad, ar.ndim, as_index=True)
    slices = tuple(slice(a, ar.shape[i] - b) for i, (a, b) in enumerate(crops))
    cropped = ar[slices]
    # mssim = crop(S, pad).mean()
    mssim = cropped.mean()

    if gradient:
        # The following is Eqs. 7-8 of Avanaki 2009.
        grad = filter_func(A1 / D, **filter_args) * X
        grad += filter_func(-S / B2, **filter_args) * Y
        grad += filter_func((ux * (A2 - A1) - uy * (B2 - B1) * S) / D,
                            **filter_args)
        grad *= (2 / X.size)

        if full:
            return mssim, grad, S
        else:
            return mssim, grad
    else:
        if full:
            return mssim, S
        else:
            return mssim

In [4]:
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

def ssim_2Dict(img_path, bg_path):
    
    ssim_dict = {'cnts_need': None, 
                 'boxes': [],
                 'points': {}
                }
    
    img_filename = os.path.basename(img_path)
    
    img = cv2.imread(img_path)
    img_bg = cv2.imread(bg_path)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_bg = cv2.cvtColor(img_bg, cv2.COLOR_BGR2GRAY)
    
    (score, diff) = ssim(gray, gray_bg, full=True)
    diff = (diff * 255).astype('uint8')
    
    blur = cv2.GaussianBlur(diff, (43, 77), 0)
    
    thresh = cv2.threshold(blur, 0, 255,
                       cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_NONE)
    cnts = grab_contours(cnts)
    cnts_need = []
    for cnt in cnts:
        area = cv2.contourArea(cnt)
        # gray1.size = gray1.width * gray1.height * gray1.channel
        # gray has only 1 channel, So area is equal with gray1.size.
        if area > gray.size * 0.03:
            cnts_need.append(cnt)
    
    ssim_dict['cnts_need'] = [i.tolist() for i in cnts_need]
    
    # Draw Boxes
    for cnt in cnts_need:
        (x, y, w, h) = cv2.boundingRect(cnt)
        ssim_dict['boxes'].append({'x': x, 'y': y,
                                   'w': w, 'h': h})
        
    # Edge
    i = 0
    for cnt in cnts_need:
        ssim_dict['points']['box_%d' % i] = []
        for point in cnt:
            ssim_dict['points']['box_%d' % i].append(point[0].tolist())
        i += 1
    
    return ssim_dict

In [5]:
bg_path = '/media/commaai-03/Data/workdata/baodi/background/rs07/1583477062.357019287.png'
test = allimgs[0]
img = os.path.join(root_dir, test)
ssim_dict = ssim_2Dict(img, bg_path)

In [6]:
ssim_dict['points'].keys()

dict_keys(['box_0', 'box_1'])

In [ ]:
bg_path = '/media/commaai-03/Data/workdata/baodi/background/rs07/1583477062.357019287.png'

for img in allimgs:
    basename = os.path.splitext(img)[0]
    img = os.path.join(root_dir, img)
    json_file = os.path.join(root_dir, basename + '.json')
    ssim_dict = ssim_2Dict(img, bg_path)
    ssim_dict['filename'] = img
    
    with open(json_file, 'r') as f:
        data = json.load(f)
        points = []
        for point in data['shapes']:
            points.append(point['points'][0])
            
        ssim_dict['head'] = points
    
    with open(json_file, 'w') as f:
        json.dump(ssim_dict, f, indent=2)
    print('[Info]: Finished %s.' % basename)

In [9]:
save_dir = '/media/commaai-03/Data/workdata/baodi/weighter/rs07/oneBox_twoHead/result'

for file in allimgs:
    basename = os.path.splitext(file)[0]
    img = os.path.join(root_dir, file)
    json_file = os.path.join(root_dir, basename + '.json')
    
    with open(json_file, 'r') as f:
        data = json.load(f)
        
    heads = data['head']
    boxes = data['boxes']
    if len(boxes) > 1:
        continue
        
    box, k, b, x, y = get_midleLine(heads, boxes)
    img = cv2.imread(img)
    drawed_img = draw_midLine(img, box, k, b, x, y)
    cv2.imshow('Result', drawed_img)
    k = cv2.waitKey(1000000)
    if k == 27 or k == ord('q'):
        cv2.destroyAllWindows()
        break
    if k == ord(' '):
        continue
        
cv2.destroyAllWindows()

In [ ]:
save_dir = '/media/commaai-03/Data/workdata/baodi/weighter/rs07/oneBox_twoHead/result'
json_file = '/media/commaai-03/Data/workdata/baodi/weighter/rs07/oneBox_twoHead/2020-03-10 17:11:52.108586-304a2653b06d-1-7.json'
img = '/media/commaai-03/Data/workdata/baodi/weighter/rs07/oneBox_twoHead/2020-03-10 17:11:52.108586-304a2653b06d-1-7.jpg'

with open(json_file, 'r') as f:
        data = json.load(f)
        
heads = data['head']
boxes = data['boxes']

In [7]:
def point_in_box(point, box):
    '''
    point: (x, y)
    box: [x, y, w, h]
    '''
    x_p, y_p = point
    x, y, w, h = box
    x_min, y_min, x_max, y_max = x, y, x+w, y+h

    if x_min > x_max:
        x_min, x_max = x_max, x_min
    elif x_min == x_max:
        raise Exception('[Error]: x_min == x_max, it is not a box.')

    if y_min > y_max:
        y_min, y_max = y_max, y_min
    elif y_min == y_max:
        raise Exception('[Error]: y_min == y_max, it is not a box.')

    x_v = (x_p >= x_min and x_p <= x_max)
    y_v = (y_p >= y_min and y_p <= y_max)
    if (x_v and y_v):
        return True
    else:
        return False

def get_midleLine(heads, boxes):
        
    x1, y1 = heads[0]
    x2, y2 = heads[1]
    box = boxes[0]
    bx, by, bw, bh = box['x'], box['y'], box['w'], box['h']
    box = [bx, by, bw, bh]
    
    # midleLine: y = kx + b
    k, b, x, y = [None] * 4
    
    if (x1 - x2) == 0 and (y1 - y2) != 0:
        y = (y1 + y2) / 2
        return box, k, b, x, y
    elif (y1 - y2) == 0 and (x1 - x2) != 0:
        x = (x1 + x2) / 2
        return box, k, b, x, y
    
    if (x1 - x2) == 0 and (y1 - y2) == 0:
        raise Exception('[Error]: x1 - x2 = 0 and y1 - y2 = 0!')
    
    # x1 - x2 != 0 and y1 - y2 != 0
    k = - (x1 - x2) / (y1 - y2)
    x_midpoint, y_midpoint = (x1 + x2) / 2, (y1 + y2) / 2
    b = y_midpoint - (k * x_midpoint)
    
    return box, k, b, x, y

In [8]:
def crop_by_midLine(img, box, k, b, x=None, y=None):

    bx, by, bw, bh = box
    if x:
        crop_img_1 = img[by:by+bh, bx:bx+(w/2)]
        crop_img_2 = img[by:by+bh, bx+(w/2):bx+w]
    if y:
        crop_img_1 = img[by:by+(bh/2), bx:bx+w]
        crop_img_2 = img[by+(bh/2):by+bh, bx:bx+w]
    if x and y:
        raise Exception('[Error]: crop only nead x or y or neither, not both.')
    
    if not x and not y:
        def filter_line(x, y, k, b):
            return (k*x + b) <= y

        crop_img_1 = np.zeros((bh, bw, 3), dtype=np.uint8)
        crop_img_2 = np.zeros((bh, bw, 3), dtype=np.uint8)

        for _w in range(bw):
            for _h in range(bh):
                x, y = bx + _w, by + _h
                if filter_line(x, y, k, b):
                    crop_img_1[_h, _w] = img[y, x]
                else:
                    crop_img_2[_h, _w] = img[y, x]
    return crop_img_1, crop_img_2


def draw_midLine(img, box, k, b, x=None, y=None):
    bx, by, bw, bh = box
    if x:
        p1 = [x, by]
        p2 = [x, by+bh]
    if y:
        p1 = [bx, y]
        p2 = [bx+bw, y]
    if x and y:
        raise Exception('[Error]: crop only nead x or y or neither, not both.')
        
    if not x and not y:
        # Dangerous: num * num != num^2
        _y1 = k * bx + b
        _p1 = [bx, _y1]
        _y2 = k * (bx + bw) + b
        _p2 = [bx+bw, _y2]
        _x1 = (by - b) / k
        _p3 = [_x1, by]
        _x2 = (by + bh - b) / k
        _p4 = [_x2, by+bh]
        all_points = [_p1, _p2, _p3, _p4]
        result = []
        for point in all_points:
            if point_in_box(point, box):
                result.append(point)       
        assert len(result) == 2, '[Error]: Midline error, one line not two points.'
        p1, p2 = result
        
    p1 = [int(i) for i in p1]
    p2 = [int(i) for i in p2]
    # box
    drawed_img = cv2.rectangle(img, (bx, by), (bx+bw, by+bh), (0, 0, 255), 2)
    # line
    drawed_img = cv2.line(drawed_img, tuple(p1), tuple(p2), (255,0,0), thickness=3)
    return drawed_img

In [ ]:
box, k, b, x, y = get_midleLine(heads, boxes)

In [ ]:
print('box:', box)
print('k:', k)
print('b:', b)
print('x:', x)
print('y:', y)

In [ ]:
img = cv2.imread(img)
drawed_img = draw_midLine(img, box, k, b, x, y)

In [ ]:
while 1:
    cv2.imshow('drawed', drawed_img)
    k = cv2.waitKey(1)
    if k == 27 or k == ord('q'):
        break
cv2.destroyAllWindows()